# Reading Data

In [1]:
# import libraries
from osgeo import ogr
import sys, os

In [2]:
# get ogr drivers
cnt = ogr.GetDriverCount()
for i in range(cnt):
    driver = ogr.GetDriver(i)
    print (driver.GetName())
    if i == 10:
        break    # limiting result to 10

ESRIC
FITS
PCIDSK
netCDF
PDS4
VICAR
JP2OpenJPEG
PDF
MBTiles
BAG
EEDA


In [3]:
# Read file and visualize geometry and attributes
fn = r'Economic_hub.shp'
ds = ogr.Open(fn, 0)
if ds is None:
    sys.exit(f'Could not open {fn}')
lyr = ds.GetLayer(0)

i = 0
for feat in lyr:
    pt = feat.geometry()
    x = pt.GetX()
    y = pt.GetY()
    name = feat.GetField("name")
    amenity = feat.GetField("amenity")
    print(f"Name: {name}, Amentity Type: {amenity}, Lat: {y}, Lng: {x}")
    i += 1
    if i == 10:
        break
del ds

Name: Club Nasha, Amentity Type: bar, Lat: 28.2107156, Lng: 83.9565679
Name: Paralounge, Amentity Type: bar, Lat: 28.2102933, Lng: 83.9569089
Name: New Sky Palace Dance Bar & Restaurant, Amentity Type: bar, Lat: 28.2127516, Lng: 83.9572634
Name: VE'YE Restro Bar Lounge, Amentity Type: bar, Lat: 28.2234303, Lng: 83.9875321
Name: Planet Purple, Amentity Type: bar, Lat: 28.2103676, Lng: 83.9558355
Name: None, Amentity Type: bar, Lat: 28.2113859, Lng: 83.9664603
Name: None, Amentity Type: bar, Lat: 28.2095861, Lng: 83.9569165
Name: None, Amentity Type: bar, Lat: 28.217041, Lng: 83.9573992
Name: None, Amentity Type: bar, Lat: 28.2085233, Lng: 83.9578373
Name: Sugar Cane Bar, Amentity Type: bar, Lat: 28.2114524, Lng: 83.9570616


In [4]:
# Get layer name, feature count, extend, geometry type
ds = ogr.Open(fn, 0)
for lyr in ds:
    print(f"Layer: {lyr.GetName()}")
    print(f"Feature Count: {lyr.GetFeatureCount()}")
    (xmin, xmax, ymin, ymax)= lyr.GetExtent()
    print(f"Extend: {xmin, xmax, ymin, ymax}")
    print(f"Geometry Type: {ogr.GeometryTypeToName(lyr.GetGeomType())}")

Layer: Economic_hub
Feature Count: 819
Extend: (83.9400659, 84.0309624, 28.1731438, 28.263442753497404)
Geometry Type: Point


In [5]:
# Get spatial reference
srs = lyr.GetSpatialRef()
if srs is not None:
    print(f"SRS: {srs.ExportToWkt()}")

SRS: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]


In [6]:
# Get layer field, type, width and precision
lyr_defn = lyr.GetLayerDefn()
for i in range(lyr_defn.GetFieldCount()):
    field_defn = lyr_defn.GetFieldDefn(i)
    name = field_defn.GetName()
    d_type = ogr.GetFieldTypeName(field_defn.GetType())
    width = field_defn.GetWidth()
    prec = field_defn.GetPrecision()
    print(f"Field: {name}, Type: {d_type}, Width: {width}, Precision:{prec}")
    if i == 10:
        break

Field: fid, Type: Real, Width: 20, Precision:0
Field: full_id, Type: String, Width: 254, Precision:0
Field: osm_id, Type: String, Width: 254, Precision:0
Field: osm_type, Type: String, Width: 254, Precision:0
Field: amenity, Type: String, Width: 254, Precision:0
Field: name, Type: String, Width: 254, Precision:0
Field: name_ne, Type: String, Width: 254, Precision:0
Field: addr_city, Type: String, Width: 254, Precision:0
Field: addr_distr, Type: String, Width: 254, Precision:0
Field: addr_stree, Type: String, Width: 254, Precision:0
Field: contact_ph, Type: String, Width: 254, Precision:0


# Writing Data

In [7]:
# getting writeable layer
fn = r'Fuel_station.shp'
driver = ogr.GetDriverByName("ESRI Shapefile") # choose appropriate driver
ids = driver.Open(fn, 0) # open file
if ids is None:
    sys.exit(f'Could not open {fn}')
inlyr = ids.GetLayer(0)

In [8]:
# Delete file if it already exits
if os.path.exists('test.shp'):
    driver.DeleteDataSource('test.shp')

# Create new file
ds = driver.CreateDataSource('test.shp')
layer = ds.CreateLayer('test', geom_type=ogr.wkbPoint)

In [9]:
# Adding fields
fldDef = ogr.FieldDefn('id', ogr.OFTInteger) # for integer
fieldDefn = ogr.FieldDefn('id', ogr.OFTString) # for string
fieldDefn.SetWidth(4)
layer.CreateField(fieldDefn) # create field

0

In [10]:
# Create field using previous file
fieldDefn = inlyr.GetFeature(0).GetFieldDefnRef('amenity')
layer.CreateField(fieldDefn)

0

In [11]:
# Create new feature object
featureDefn = layer.GetLayerDefn()
feature = ogr.Feature(featureDefn)

In [12]:
# Set geometry for new feature
point=ogr.Geometry(ogr.wkbPoint)
point.SetPoint(0,10,10)
feature.SetGeometry(point)

# Set attributes
feature.SetField('id', 23)

# Write feature to layer
layer.CreateFeature(feature)
ds.Destroy()

In [13]:
# View layer details using geopandas
import geopandas as gpd
gfd = gpd.read_file("test.shp")
gfd.head()

,id,amenity,geometry
0,23,None,POINT (10.00000 10.00000)


In [18]:
from osgeo import ogr, osr
import os

In [19]:
driver = ogr.GetDriverByName('ESRI Shapefile')
dataset = driver.Open(r"fountain.shp", 0)
layer = dataset.GetLayer(0)
feature = layer.GetNextFeature()
geom = feature.GetGeometryRef()
print(geom.GetX(), geom.GetY())

83.97709634943661 28.255935984288143


In [20]:
geoSR = osr.SpatialReference()
geoSR.ImportFromEPSG(4326)

utmSR = osr.SpatialReference()
utmSR.ImportFromEPSG(32644)

coordTrans = osr.CoordinateTransformation(geoSR, utmSR)
geom.Transform(coordTrans)

print(geom.GetX(), geom.GetY())

-35473.97578762239 9589941.13456382
